Import the libraries for the script

In [3]:
import findspark

findspark.init()
import pyspark
import pyspark.sql
from pyspark import SparkConf
from pyspark.sql import SparkSession
fr import Statistics

spark = SparkSession.builder.getOrCreate()
SparkConf().getAll()

[(u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.app.name', u'pyspark-shell')]

Import the data from local csv files

In [4]:
media = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load(path='C:\Users\ChrisGomes\Projects\MediaStock\gdelt\*.csv')
sp500 = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load(path='C:\Users\ChrisGomes\Projects\MediaStock\s&p500.csv')
sp500Companies = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load(path='C:\Users\ChrisGomes\Projects\MediaStock\s&p500Companies.csv')


Check correlation between average GoldsteinScale and price change per day

In [23]:
# get avg goldstein scale per date
goldsteinScales = media.groupby('SQLDATE').agg(pyspark.sql.functions.avg('GoldsteinScale'))


In [24]:
# calculate absolute price change
sp500 = sp500.withColumn('PriceChange', pyspark.sql.functions.abs(sp500['Open'] - sp500['Close']))

# Convert Date to SQLDATE format
sp500 = sp500.withColumn('SQLDATE', pyspark.sql.functions.date_format('Date', 'yyyyMMdd'))

# get PriceChange per date
priceChanges = sp500.groupby('SQLDATE').agg(pyspark.sql.functions.avg('PriceChange'))

In [ ]:
# calculate correlation
goldsteinAndPrice = goldsteinScales.join(priceChanges)

AnalysisException: u'Detected cartesian product for INNER join between logical plans\nAggregate [SQLDATE#13], [SQLDATE#13, avg(GoldsteinScale#42) AS avg(GoldsteinScale)#406]\n+- Project [SQLDATE#13, GoldsteinScale#42]\n   +- Relation[GLOBALEVENTID#12,SQLDATE#13,MonthYear#14,Year#15,FractionDate#16,Actor1Code#17,Actor1Name#18,Actor1CountryCode#19,Actor1KnownGroupCode#20,Actor1EthnicCode#21,Actor1Religion1Code#22,Actor1Religion2Code#23,Actor1Type1Code#24,Actor1Type2Code#25,Actor1Type3Code#26,Actor2Code#27,Actor2Name#28,Actor2CountryCode#29,Actor2KnownGroupCode#30,Actor2EthnicCode#31,Actor2Religion1Code#32,Actor2Religion2Code#33,Actor2Type1Code#34,Actor2Type2Code#35,... 34 more fields] csv\nand\nAggregate [SQLDATE#426], [SQLDATE#426, avg(PriceChange#415) AS avg(PriceChange)#450]\n+- Project [abs((Open#142 - Close#145)) AS PriceChange#415, date_format(Date#141, yyyyMMdd, Some(America/New_York)) AS SQLDATE#426]\n   +- Relation[Date#141,Open#142,High#143,Low#144,Close#145,Adj Close#146,Volum